# Traducción de preguntas y respuestas

En este notebook se realiza la traducción de preguntas y respuestas del dataset HeadQA de español a inglés con GPT-4o-mini, con el fin de utilizar un modelo decoder preentrenado en inglés en el ámbito biomédico.
El notebook donde se hace uso de la traducción es `decoders_mca.ipynb`, este es simplemente un notebook de apoyo para la traducción.

In [1]:
!pip install openai python-dotenv --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
e2b 1.0.5 requires packaging>=24.1, but you have packaging 23.2 which is incompatible.
googletrans 4.0.0rc1 requires httpx==0.13.3, but you have httpx 0.28.1 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
from openai import AzureOpenAI
from dotenv import load_dotenv
import os
import time
import pandas as pd

Previamente se cargaron las variables de entrono necesarias para hacer uso de la API de OpenAI. Para replicarlo, se debe guardar un archivo `.env` con las variables `AZURE_OPENAI_KEY`, `AZURE_ENDPOINT`, `DEPLOYMENT`, `MODEL_NAME` Y `API_VERSION`.

In [3]:
# Cargar variables de entorno
load_dotenv(override=True)
endpoint = os.getenv("AZURE_ENDPOINT")
model_name = os.getenv("deployment")
deployment = os.getenv("DEPLOYMENT")

subscription_key = os.getenv("AZURE_OPENAI_KEY")
# api_version = os.getenv("API_VERSION")
api_version = "2025-03-01-preview"

# Crear cliente de AzureOpenAI
client = AzureOpenAI(
    api_version=api_version,
    azure_endpoint=endpoint,
    api_key=subscription_key
)

Se hace una prueba de conceepto traduciendo un texto de ejemplo.

In [4]:
response = client.chat.completions.create(
    model=deployment,
    messages=[
        {"role": "system", "content": "Traduce de español a ingles."},
        {"role": "user", "content": "Tengo un aneurisma."}
    ],
    max_tokens=4096,
    temperature=1.0,
    top_p=1.0,
)

print(response.choices[0].message.content)


I have an aneurysm.


Una vez comprobado el funcionamiento de la traducción, procedemos a traducir todas las preguntas y respuestas del dataset de test.

In [ ]:
test_df = pd.read_csv('data/test.csv', index_col=0)

In [48]:
test_df.head()

,qid,qtext,ra,image,answer_1,answer_2,answer_3,answer_4,answer_5
4039,87,El virión de los retrovirus:,4,NaN,Tiene forma helicoidal.,Tiene forma icosaédrica.,Contiene una sola copia de su genoma.,Contiene dos copias de su genoma.,Contiene un genoma segmentado.
4173,224,El suelo de la cavidad amniótica es el:,3,NaN,Trofoblasto.,Hipoblasto.,Epiblasto.,Endometrio.,Miometrio.
3975,18,Las señales sensitivas llegan principalmente a...,3,NaN,II.,III.,IV.,V.,VI.
2470,25,Las desviaciones instrumentales de la Ley de B...,2,NaN,Variaciones en la temperatura lo que provoca d...,Empleo de radiación no monocromática y presenc...,"Empleo de concentraciones elevadas de analito,...",La participación de la especie absorbente en u...,Respuesta incorrecta
2255,36,La presencia de síntomas o déficits que afecta...,3,NaN,Trastorno somatomorfo indiferenciado.,Trastorno de somatización.,Trastorno de conversión.,Trastorno por dolor.,Respuesta incorrecta


Preparamos los datasets para su traducción

In [ ]:
# columnas a traducir
cols_a_traducir = ["qtext", "answer_1", "answer_2", "answer_3", "answer_4", "answer_5"]

# inicializar nuevas columnas traducidas
for col in cols_a_traducir:
    test_df[col + "_en"] = ""

Se crea una función que recibe un batch a traducir (lista de textos).

In [ ]:
def traducir_batch(lista_textos: list[str]) -> list[str]:
    """
    Traduce una lista de textos del español al inglés usando Azure OpenAI GPT-4o-mini.
    Args:
        lista_textos (list[str]): Lista de textos en español a traducir.
    Returns:
        list[str]: Lista de textos traducidos al inglés.
    """
    # Manejo de respuestas vacías
    cleaned = ["(VACÍO)" if (pd.isna(t) or not isinstance(t, str) or t.strip() == "") else t 
               for t in lista_textos]

    # Construir prompt
    prompt = (
        "Traduce a inglés cada uno de estos textos en el ámbito médico/biomédico.\n"
        "Responde solo con una lista, cada ítem en una nueva línea, en el MISMO orden que se muestran:\n\n"
    )
    for t in cleaned:
        prompt += f"- {t}\n"

    try:
        respuesta = client.chat.completions.create(
            model=deployment,   # usa tu deployment que sí funciona
            messages=[
                {"role": "system", "content": "Eres un traductor biomédico profesional."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=4096,
            temperature=0.0
        )

        texto = respuesta.choices[0].message.content

        # dividir líneas y limpiar
        output = [
            line.lstrip("-•* ").strip()
            for line in texto.split("\n")
            if line.strip() != ""
        ]

        # revertir "(VACÍO)" a ""
        output = ["" if t in ["(VACÍO)", "(EMPTY)"] else t for t in output]

        # asegurar longitud correcta
        if len(output) != len(lista_textos):
            print("Advertencia: el modelo devolvió un número diferente de líneas.")
            while len(output) < len(lista_textos):
                output.append("")
            output = output[:len(lista_textos)]

        return output

    except Exception as e:
        print("Error traduciendo batch:", e)
        return [""] * len(lista_textos)


Se crea una función para traducir un DataFrame completo usando batches.

In [ ]:
def traducir_df(df:pd.DataFrame, cols:list[str], batch_size:int=20, delay:float=0.3) -> pd.DataFrame:
    """
    Traduce las columnas especificadas de un DataFrame en batches.
    Args:
        df (pd.DataFrame): DataFrame con las columnas a traducir.
        cols (list[str]): Lista de nombres de columnas a traducir.
        batch_size (int): Tamaño de cada batch para la traducción.
        delay (float): Tiempo de espera entre batches para evitar rate limits.
    Returns:
        pd.DataFrame: DataFrame con las columnas traducidas añadidas.
    """
    for col in cols:
        print(f"Traduciendo columna: {col}")
        textos = df[col].tolist()
        traducciones = []

        for i in range(0, len(textos), batch_size):
            batch = textos[i:i + batch_size]
            print(f"  Traduciendo batch {i // batch_size + 1}...")
            traducciones_batch = traducir_batch(batch)
            traducciones.extend(traducciones_batch)
            time.sleep(delay)  # esperar para evitar rate limits

        df[col + "_en"] = traducciones
        print(f"Columna {col} traducida.")

    return df[['qid', 'qtext_en', 'ra', 'answer_1_en', 'answer_2_en', 'answer_3_en', 'answer_4_en', 'answer_5_en']]

In [ ]:
# prueba con tres filas
test_sample = test_df.head(2).copy()
test_sample_traducido = traducir_df(test_sample, cols_a_traducir, batch_size=2, delay=0.01)
test_sample_traducido.head()

Traduciendo columna: qtext
  Traduciendo batch 1...
  Traduciendo batch 2...
Columna qtext traducida.
Traduciendo columna: answer_1
  Traduciendo batch 1...
  Traduciendo batch 2...
Columna answer_1 traducida.
Traduciendo columna: answer_2
  Traduciendo batch 1...
  Traduciendo batch 2...
Columna answer_2 traducida.
Traduciendo columna: answer_3
  Traduciendo batch 1...
  Traduciendo batch 2...
Columna answer_3 traducida.
Traduciendo columna: answer_4
  Traduciendo batch 1...
  Traduciendo batch 2...
Columna answer_4 traducida.
Traduciendo columna: answer_5
  Traduciendo batch 1...
  Traduciendo batch 2...
Columna answer_5 traducida.


,qid,qtext,ra,image,answer_1,answer_2,answer_3,answer_4,answer_5,qtext_en,answer_1_en,answer_2_en,answer_3_en,answer_4_en,answer_5_en
2708,34,Activa el sistema renina-angiotensinaal...,5,NaN,Presión arterial.,Actividad parasimpática.,Concentración de Ca2+ en plasma.,Concentración de Na+ en plasma.,Actividad simpática.,Activates the renin-angiotensin-aldosterone sy...,Blood pressure.,Parasympathetic activity.,Plasma Ca2+ concentration.,Plasma Na+ concentration.,Sympathetic activity.
3774,46,La cromatografía de gases combinada con la esp...,5,NaN,El análisis de iones moleculares en el mismo s...,La producción de iones muy fragmentados con ge...,El aumento de la velocidad de flujo de salida ...,La obtención del espectro bidimensional que pe...,La introducción en el alto vacío del analizado...,Gas chromatography combined with mass spectrom...,The analysis of molecular ions at the same sit...,The production of highly fragmented ions with ...,The increase in the outflow rate of the chroma...,The acquisition of the two-dimensional spectru...,The introduction into the high vacuum of the m...
3391,115,"Hombre de 50 años, bronquítico crónico que ing...",4,NaN,Cambiaría el tratamiento a ceftriaxona por su ...,Añadiría al tratamiento una quinolona.,Cambiaría a amoxicilina / clavulánico.,Descartaría la presencia de un empiema pleural.,"Seguiría con el mismo tratamiento, suponiendo ...","50-year-old man, chronic bronchitic, admitted ...",I would change the treatment to ceftriaxone du...,I would add a quinolone to the treatment.,I would switch to amoxicillin/clavulanate.,I would rule out the presence of a pleural emp...,"I would continue with the same treatment, assu..."


Finalmente se traduce el dataset de test completo.

In [ ]:
# Funciona bien :D pasamos el df completo
print(f'longitud test_df: {len(test_df)}')
translated_test_df = traducir_df(test_df, cols_a_traducir, batch_size=30, delay=0.01)
print('Translated test')

longitud test_df: 790
Traduciendo columna: qtext
  Traduciendo batch 1...
  Traduciendo batch 2...
  Traduciendo batch 3...
  Traduciendo batch 4...
  Traduciendo batch 5...
  Traduciendo batch 6...
  Traduciendo batch 7...
  Traduciendo batch 8...
  Traduciendo batch 9...
  Traduciendo batch 10...
  Traduciendo batch 11...
  Traduciendo batch 12...
  Traduciendo batch 13...
  Traduciendo batch 14...
  Traduciendo batch 15...
  Traduciendo batch 16...
  Traduciendo batch 17...
  Traduciendo batch 18...
  Traduciendo batch 19...
  Traduciendo batch 20...
  Traduciendo batch 21...
  Traduciendo batch 22...
  Traduciendo batch 23...
  Traduciendo batch 24...
  Traduciendo batch 25...
  Traduciendo batch 26...
  Traduciendo batch 27...
Columna qtext traducida.
Traduciendo columna: answer_1
  Traduciendo batch 1...
  Traduciendo batch 2...
  Traduciendo batch 3...
  Traduciendo batch 4...
  Traduciendo batch 5...
  Traduciendo batch 6...
  Traduciendo batch 7...
  Traduciendo batch 8...
  T

In [ ]:
# guardar los resultados de cada df
translated_test_df.to_csv('data/test_en.csv')